In [23]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

with open("./Actas/ActaLegaPeñaAmistad.html") as fp:
    soup = BeautifulSoup(fp)

# Obtener DataFrames y guardarlos en Excel

In [24]:
x = [x.text for x in soup.find_all('td',attrs={"no wrap": ""})]
texto=str(x)
eventos=texto.split("\'")
local=eventos[1]
visitante=eventos[5]

cuerpo=soup.body
central=cuerpo.find_all("div",class_="page-wrapper-row full-height")
contenido=central[0].find_all("div",class_="page-content")
contenido=contenido[0].find_all("div",class_="row nova_text")

centro=contenido[0].find_all("div",class_="col-sm-4 col-sm-push-4")
izqda=contenido[0].find_all("div",class_="col-sm-4 col-sm-pull-4")
dcha=contenido[0].find_all("div",class_="col-sm-4")[2]

#CENTRO (EQUIPOS Y RESULTADO)
contcentro=centro[0].find_all("tr")
equipos=contcentro[0].find_all("td")
local=equipos[0].get_text()
visitante=equipos[-1].get_text()
tablagoles=centro[0].find_all("table")[1]
goles=tablagoles.find_all("tr")
resultado=goles[-1].find_all("strong")[0].get_text().split('\xa0\xa0\xa0')[1].replace(" ","")
goleslocal=int(resultado.split('-')[0])
golesvisitante=int(resultado.split('-')[1])

golestotales=goleslocal+golesvisitante

listagoles=[]
if golestotales==0:
    listagoles=["0,0",0,0]
else:
    for i in range(13,(13+3*golestotales),3):
        resultadogol=eventos[i][-7:]
        resultadogol=resultadogol.replace(" ","")
        if len(eventos[i+1])==6:
            minutogol=int(eventos[i+1][4:6])
        else:
            minutogol=int(eventos[i+1][4])
        goleadora=eventos[i+2].split(') ')[1].split(' "')[0]
        a=(resultadogol,minutogol,goleadora)
        listagoles.append(a)
dfgoles=pd.DataFrame(listagoles,columns=['Gol','Minuto','Jugadora'])

#IZQUIERDA (TITULARES,SUPLENTES Y SUSTITUCIONES)
primernivelloc=izqda[0].find_all("div",class_="desc")
tablatitularesloc=primernivelloc[0].find_all("table")[0]
tablasuplentesloc=primernivelloc[0].find_all("table")[1]
titularesloc=len(tablatitularesloc.find_all("tr"))
suplentesloc=len(tablasuplentesloc.find_all("tr"))
sustloc=len(primernivelloc[0].find_all("table"))-3

#DERECHA (TITULARES, SUPLENTES Y SUSTITUCIONES)
primernivelvis=dcha.find_all("div",class_="desc")
tablatitularesvis=primernivelvis[0].find_all("table")[0]
tablasuplentesvis=primernivelvis[0].find_all("table")[1]
titularesvis=len(tablatitularesvis.find_all("tr"))
suplentesvis=len(tablasuplentesvis.find_all("tr"))
sustvis=len(primernivelvis[0].find_all("table"))-3

#ALINEACIONES Y MINUTOS JUGADOS

txtali=eventos[((13+3*golestotales)+16):]

#Local

totalesloc=titularesloc+suplentesloc
txtjugloc=txtali[0:(4*totalesloc)]
listalocal=[]
    #paralastitulares
for i in range(2,2+4*(titularesloc),4):
    jugadora=txtjugloc[i]
    situacion="titular"
    a=(jugadora,situacion)
    listalocal.append(a)
    
    #suplentes
for i in range(2+4*titularesloc,len(txtjugloc),4):
    jugadora=txtjugloc[i]
    situacion="suplente"
    a=(jugadora,situacion)
    listalocal.append(a)
dfjugloc=pd.DataFrame(listalocal,columns=['Jugadora','Situación'])

dftitloc=dfjugloc.head(titularesloc)
dfsuploc=dfjugloc.tail(suplentesloc)

#Sustituciones
txtloc=txtali[0:(4*totalesloc+11*sustloc)]
txtsusloc=txtloc[4*totalesloc:(4*totalesloc)+(11*sustloc)]
listasusloc=[]
for i in range(2,len(txtsusloc),11):
    entra=txtsusloc[i+6].split(') ')[1].split(' "')[0]
    sale=txtsusloc[i]
    if len(txtsusloc[i+5])==6:
        minuto=int(txtsusloc[i+5][4:6])
    else:
        minuto=int(txtsusloc[i+5][4])
    a=(entra,sale,minuto)
    listasusloc.append(a)
dfsusloc=pd.DataFrame(listasusloc,columns=['entra','Jugadora','minuto'])

dfmergedsusloc=dfsuploc.merge(dfsusloc,how='outer',on='Jugadora')
dfmergedsusloc.drop(columns=['entra'], inplace=True)
dfmergedsusloc=dfmergedsusloc.rename(columns={'minuto': 'minutosjugados'})
dfmergedsusloc.fillna(90, inplace=True)
dfmergedsusloc['minutosjugados']=90-dfmergedsusloc['minutosjugados']

dfsuslocprov=dfsusloc.rename(columns={'Jugadora': 'sale','entra':'Jugadora'})

dfmergedtitloc=dftitloc.merge(dfsuslocprov, how='outer',on='Jugadora')
dfmergedtitloc.drop(columns=['sale'], inplace=True)
dfmergedtitloc=dfmergedtitloc.rename(columns={'minuto': 'minutosjugados'})
dfmergedtitloc.fillna(90, inplace=True)

dfbuenoloc=pd.concat([dfmergedtitloc,dfmergedsusloc],sort=False)

##EQUIPO VISITANTE
totalesvis=titularesvis+suplentesvis
txtvis=txtali[len(txtloc):]
txtjugvis=txtvis[0:(4*totalesvis)]
listavisitante=[]
    #paralastitulares
for i in range(2,2+4*(titularesvis),4):
    jugadora=txtjugvis[i]
    situacion="titular"
    a=(jugadora,situacion)
    listavisitante.append(a)
    
    #suplentes
for i in range(2+4*titularesvis,len(txtjugvis),4):
    jugadora=txtjugvis[i]
    situacion="suplente"
    a=(jugadora,situacion)
    listavisitante.append(a)
dfjugvis=pd.DataFrame(listavisitante,columns=['Jugadora','Situación'])

#Dos df, uno para titulares y otro para visitantes
dftitvis=dfjugvis.head(titularesvis)
dfsupvis=dfjugvis.tail(suplentesvis)

#Sustituciones

txtsusvis=txtvis[4*totalesvis:(4*totalesvis)+(11*sustvis)]
listasusvis=[]
for i in range(2,len(txtsusvis),11):
    entra=txtsusvis[i+6].split(') ')[1].split(' "')[0]
    sale=txtsusvis[i]
    if len(txtsusvis[i+5])==6:
        minuto=int(txtsusvis[i+5][4:6])
    else:
        minuto=int(txtsusvis[i+5][4])
    a=(entra,sale,minuto)
    listasusvis.append(a)
dfsusvis=pd.DataFrame(listasusvis,columns=['entra','Jugadora','minuto'])

dfmergedsupvis=dfsupvis.merge(dfsusvis,how='outer',on='Jugadora')
dfmergedsupvis.drop(columns=['entra'], inplace=True)
dfmergedsupvis=dfmergedsupvis.rename(columns={'minuto': 'minutosjugados'})
dfmergedsupvis.fillna(90, inplace=True)
dfmergedsupvis['minutosjugados']=90-dfmergedsupvis['minutosjugados']

dfsusvisprov=dfsusvis.rename(columns={'Jugadora': 'sale','entra':'Jugadora'})

dfmergedtitvis=dftitvis.merge(dfsusvisprov, how='outer',on='Jugadora')
dfmergedtitvis.drop(columns=['sale'], inplace=True)
dfmergedtitvis=dfmergedtitvis.rename(columns={'minuto': 'minutosjugados'})
dfmergedtitvis.fillna(90, inplace=True)

dfbuenovis=pd.concat([dfmergedtitvis,dfmergedsupvis],sort=False)

writer = pd.ExcelWriter('./DatosActas/Datos{}-{}.xlsx'.format(local,visitante), engine='xlsxwriter')

dfbuenoloc.to_excel(writer, sheet_name='Sheet1', index=False, startrow=1, startcol=1)
dfsusloc.to_excel(writer, sheet_name='Sheet1',index=False, startrow=1, startcol=5)
dfbuenovis.to_excel(writer,sheet_name='Sheet1',index=False, startrow=19, startcol=1)
dfsusvis.to_excel(writer,sheet_name='Sheet1',index=False,startrow=19, startcol=5)
dfgoles.to_excel(writer, sheet_name='Sheet1',index=False, startrow=1, startcol=9)

writer.save()